気象データを特徴量としてpredNetに入れられるよう前処理

In [ ]:
import numpy as np
import shutil
import gzip
import os
import pandas as pd
import datetime as dt
import hickle as hkl

# gzip を解凍し、バイナリデータを読みこむ
def Read_gz_Binary(file):
    file_tmp = file + "_tmp"
    with gzip.open(file, 'rb') as f_in:
        with open(file_tmp, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)    
    bin_data = np.fromfile(file_tmp, np.float32)
    os.remove(file_tmp)
    return bin_data.reshape( [168,128] )

# met データの欠損処理
def fill_lack_data(data):
    # まず欠損行の穴埋めは、値が存在する上下端の行の値をそのままコピーする
    data[0:2] = data[2]
    data[154:] = data[153]
    # 欠損列の穴埋めも、値が存在する左右端の列の値をそのままコピーする
    data[:, :8] = data[:, 8].reshape(-1,1)
    return data

# パスのデータの欠損処理
def missing_fix(path):
    data = Read_gz_Binary(path)
    data = fill_lack_data(data)
    
    return data

# 湿数の生成
def gen_TTd(P, T, H):
    '''
    P: 気圧[hPa]
    T: 温度[K]
    H: 湿度[%]
    '''
    # 飽和蒸気圧[hPa]
    Pes = 6.112 * np.exp( 17.67 * (T-273.15) / (T-29.65) )
    # 蒸気圧[hPa]
    Pe = H/100 * Pes
    # 混合比[kg/kg]
    MR = 0.0006112 * P/(P-Pe)
    # 露点温度[K]
    Td = (29.65*np.log(Pe/6.112) - 17.67*273.15)/(np.log(Pe/6.112) - 17.67)
    # 湿数[K]
    TTd = T - Td
    return TTd

# 湿数画像 （二次配列） を生成
def gen_TTd_mat(row, border, pressure):
    #ファイルが存在しなかったら一つ前を参照 (存在するまで繰り返す)
    while True:
        try:
            # 湿度と温度のデータ取得( 上下左の欠損処理も込み )
            RH_data = missing_fix(df['rh_path'][row])
            TM_data = missing_fix(df['tmp_path'][row])
            break
        except FileNotFoundError:
            row-=1
    # 湿数データを計算
    TTd_data = gen_TTd(pressure, TM_data, RH_data)
    # 最小値0, 最大値255に変換
    TTd_data = (TTd_data - np.min(TTd_data))/(np.max(TTd_data)-np.min(TTd_data)) * 255
    # 雲の存在可能性と湿数のあたいは正負逆の関係にあるので、グレースケール反転
    TTd_data = 255 - TTd_data
    # 閾値以下を 0 にする
    TTd_data = TTd_data * (TTd_data > border)
    # 上8行を削る
    TTd_data = np.split(TTd_data, [8])[1]
    # サイズを縦横各 4 倍　(雲画像と同じサイズにする)
    TTd_data = TTd_data.repeat(4,axis=0).repeat(4,axis=1)
    return TTd_data


# 湿度・温度 (500hPa) のパス csv データ読み込み
df = pd.read_csv('../data/external/test_rh_tmp_500_path_info.csv', index_col=0)
# 不要部分カット
df = df[df.index>12].reset_index(drop=True)

# 全時刻での湿数画像 (二次配列) 群のmap生成( key : 日付 ( datetime 型), value : 画像 )
alltime_TTd_map = {}
for row in range(df.shape[0]):
    tmp_date = dt.datetime.strptime(df['rh_file_name'][row][9:-3], '%Y%m%d%H')
    alltime_TTd_map[tmp_date] = gen_TTd_mat(row, 180, 500)
    tmp_date = dt.datetime.strptime(df['rh_file_name'][row][9:-3], '%Y%m%d%H') + dt.timedelta(hours=1)
    alltime_TTd_map[tmp_date] = gen_TTd_mat(row, 180, 500)
    tmp_date = dt.datetime.strptime(df['rh_file_name'][row][9:-3], '%Y%m%d%H') + dt.timedelta(hours=2)
    alltime_TTd_map[tmp_date] = gen_TTd_mat(row, 180, 500)

# # 入力画像群の各終了時刻 (文字列) の取得
term_df = pd.read_csv('../data/input/inference_terms.csv')
end_time_sr = term_df['OpenData_96hr_End']
# 開始時刻の導出 ( window = 入力枚数 )
window = 24
start_time_list = []
for i in range(len(end_time_sr)):
    # 文字列->datetime型
    end_time = dt.datetime.strptime(end_time_sr[i], '%Y/%m/%d %H:%M')
    start_time = end_time - dt.timedelta(hours = window-1)
    start_time_list.append(start_time)

# # 入力四次配列生成
input_data_list = []
for j in range(len(start_time_list)):
    # 入力セットの生成
    start_time = start_time_list[j]
    input_set_list = []
    for i in range(window):
        input_set_list.append(alltime_TTd_map[start_time+dt.timedelta(hours=i)])
    # 0 埋め画像 24 枚追加 (出力用)
    for i in range(window):
        input_set_list.append(np.zeros_like(alltime_TTd_map[start_time]))
    input_data_list.append(input_set_list)

input_data = np.array(input_data_list)

hkl.dump( input_data, '../data/features/TTd_X_test.hkl' )